In [0]:
# this notebook is based off of this blog post: 
# https://realpython.com/python-keras-text-classification/#reader-comments

In [0]:
DATASET_NAME = "test"
MODEL_NAME = "test"
MAX_SEQ_LENGTH = 20
TRAINING_SET_SIZE = 1000000
VAL_SET_SIZE = 1000000
NUM_EPOCHS = 15
BATCH_SIZE = 100

In [62]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import hstack, vstack
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential, layers
from keras.utils import plot_model
import pandas as pd
import numpy as np
import copy
import time
import pickle
!pip install gcsfs

pd.set_option('max_colwidth', 100)

In [5]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Updated property [core/project].
gs://fb-congressional-data/dev.csv
gs://fb-congressional-data/facebook_congress_posts.csv.zip
gs://fb-congressional-data/facebook_congress_responses.csv.zip
gs://fb-congressional-data/test.csv
gs://fb-congressional-data/train.csv


In [8]:
train_df = pd.read_csv("gs://fb-congressional-data/train.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [9]:
dev_df = pd.read_csv("gs://fb-congressional-data/dev.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
# shuffle the data
# be sure to do this before you extract X's and y's!!
train_df = train_df.sample(frac=1)
dev_df = dev_df.sample(frac=1)

In [0]:
y_train = train_df.op_gender.values
y_dev = dev_df.op_gender.values

In [0]:
def turn_to_ints(li):
    final_list = []
    for gender in li:
        if gender=='M':
            final_list.append(1)
        else:
            final_list.append(0)
    return final_list
            
y_train = turn_to_ints(y_train)
y_dev = turn_to_ints(y_dev)

In [0]:
y_train = np.asarray(y_train)
y_dev = np.asarray(y_dev)

In [0]:
def get_text_list(init_list):
    sentences = []
    for sentence in init_list:
        if type(sentence) != str:
            sentences.append("")
        else:
            sentences.append(sentence)
    return sentences

new_sentences_train = get_text_list(train_df.response_text.values)
new_sentences_test = get_text_list(dev_df.response_text.values)

In [17]:
time_start = time.time()

tokenizer = Tokenizer(num_words=200000)
tokenizer.fit_on_texts(new_sentences_train)

X_train = tokenizer.texts_to_sequences(new_sentences_train)
X_test = tokenizer.texts_to_sequences(new_sentences_test)

vocab_size = len(tokenizer.word_index) + 1

currentTime = time.gmtime(time.time() - time_start)

#Convert the gmtime struct to a string
timeStr = time.strftime("%M minutes, %S seconds", currentTime)

print("Tokenized in {}".format(timeStr))

Tokenized in 10 minutes, 31 seconds


In [0]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 20

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [0]:
x_train_path = 'X_train_{}.pkl'.format(DATASET_NAME)
x_dev_path = 'X_dev_{}.pkl'.format(DATASET_NAME)
y_train_path = 'y_train_{}.pkl'.format(DATASET_NAME)
y_dev_path = 'y_dev_{}.pkl'.format(DATASET_NAME)

with open(x_train_path, 'wb') as file:
    pickle.dump(X_train, file)   
with open(x_dev_path, 'wb') as file:
    pickle.dump(X_test, file)
with open(y_train_path, 'wb') as file:
    pickle.dump(y_train, file)
with open(y_dev_path, 'wb') as file:
    pickle.dump(y_dev, file)

In [73]:
## TODO: figure out how to make these copy-able 
!gsutil cp 'X_train_test.pkl' "gs://fb-congressional-data/"
!gsutil cp {x_dev_path} gs://fb-congressional-data/{x_dev_path}
!gsutil cp {y_train_path} gs://fb-congressional-data/{y_train_path}
!gsutil cp {y_dev_path} gs://fb-congressional-data/{y_dev_path}

Copying file://X_train_test.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

-
Operation completed over 1 objects/753.7 MiB.                                    
CommandException: No URLs matched: {x_dev_path}
CommandException: No URLs matched: {y_train_path}
CommandException: No URLs matched: {y_dev_path}


In [69]:
!ls

adc.json     X_dev_test.pkl  X_train_test.pkl  y_train_test.pkl
sample_data  X_train.pkl     y_dev_test.pkl


In [0]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

embedding_dim = 100
embedding_matrix = create_embedding_matrix(
                     '/tmp/glove.6B.{}d.txt'.format(embedding_dim),
                      tokenizer.word_index, embedding_dim)

In [49]:
# hmmmm....
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size

0.15381079935942088

In [51]:
embedding_matrix.shape

(1113992, 100)

In [52]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 100)           111399200 
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 16, 128)           64128     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 111,464,629
Trainable params: 65,429
Non-trainable params: 111,399,200
_________________________________________________________________


In [0]:
smaller_X_train = X_train[:TRAINING_SET_SIZE]

In [0]:
smaller_y_train = y_train[:TRAINING_SET_SIZE]

In [0]:
smaller_X_dev = X_test[:VAL_SET_SIZE]
smaller_y_dev = y_dev[:VAL_SET_SIZE]

In [57]:
try:
  time_start = time.time()

  history = model.fit(smaller_X_train, smaller_y_train,
                      epochs=NUM_EPOCHS,
                      verbose=True,
                      validation_data=(smaller_X_dev, smaller_y_dev),
                      batch_size=BATCH_SIZE)

  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))

except:
  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))  

Train on 1000000 samples, validate on 200000 samples
Epoch 1/15
1000000/1000000 [==============================] - 285s 285us/sample - loss: 0.6019 - acc: 0.7110 - val_loss: 0.4792 - val_acc: 0.8508
Epoch 2/15
1000000/1000000 [==============================] - 280s 280us/sample - loss: 0.6012 - acc: 0.7110 - val_loss: 0.4773 - val_acc: 0.8508
Epoch 3/15
1000000/1000000 [==============================] - 279s 279us/sample - loss: 0.6008 - acc: 0.7110 - val_loss: 0.4715 - val_acc: 0.8508
Epoch 4/15
1000000/1000000 [==============================] - 279s 279us/sample - loss: 0.6003 - acc: 0.7110 - val_loss: 0.4759 - val_acc: 0.8508
Epoch 5/15
1000000/1000000 [==============================] - 281s 281us/sample - loss: 0.5997 - acc: 0.7110 - val_loss: 0.4718 - val_acc: 0.8508
Epoch 6/15
  17040/1000000 [..............................] - ETA: 4:21 - loss: 0.6017 - acc: 0.7075

KeyboardInterrupt: ignored

In [63]:
loss, accuracy = model.evaluate(smaller_X_train, smaller_y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_dev, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 0.7110
Testing Accuracy:  0.8497


In [64]:
# this will only work if you finish the training...hmm 
plot_model(history)

NameError: ignored